In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "mean"
fs_method, fs_ratio = "fcls", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9158 | AUPR: 0.6542 | Acc: 95.32%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9195 | AUPR: 0.6639 | Acc: 95.37%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9220 | AUPR: 0.6744 | Acc: 95.35%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9165 | AUPR: 0.6541 | Acc: 95.29%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9199 | AUPR: 0.6649 | Acc: 95.35%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9215 | AUPR: 0.6719 | Acc: 95.41%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9220 | AUPR: 0.6744 | Acc: 95.35%
Internal test: AUC: 0.9296 | AUPR: 0.7047 | Acc: 95.45%
External test: AUC: 0.9247 | AUPR: 0.6746 | Acc: 95.17%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9162 | AUPR: 0.6218 | Acc: 94.96%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9169 | AUPR: 0.6145 | Acc: 94.88%
model(kernel=linear, C=1)
Valid:         AUC: 0.9164 | AUPR: 0.6101 | Acc: 94.85%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9169 | AUPR: 0.6145 | Acc: 94.88%
Internal test: AUC: 0.9228 | AUPR: 0.6678 | Acc: 94.84%
External test: AUC: 0.9166 | AUPR: 0.6000 | Acc: 94.54%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7588 | AUPR: 0.4347 | Acc: 94.82%
model(n_neighbors=5)
Valid:         AUC: 0.7783 | AUPR: 0.4814 | Acc: 95.10%
model(n_neighbors=7)
Valid:         AUC: 0.7948 | AUPR: 0.4978 | Acc: 95.12%
best model(n_neighbors=7)
Valid:         AUC: 0.7948 | AUPR: 0.4978 | Acc: 95.12%
Internal test: AUC: 0.7903 | AUPR: 0.5099 | Acc: 94.82%
External test: AUC: 0.8037 | AUPR: 0.5047 | Acc: 94.88%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8730 | AUPR: 0.4009 | Acc: 85.99%
best model()
Valid:         AUC: 0.8730 | AUPR: 0.4009 | Acc: 85.99%
Internal test: AUC: 0.8842 | AUPR: 0.4318 | Acc: 85.86%
External test: AUC: 0.8708 | AUPR: 0.3782 | Acc: 84.46%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9178 | AUPR: 0.6507 | Acc: 95.24%
model(C=10.0)
Valid:         AUC: 0.9176 | AUPR: 0.6500 | Acc: 95.21%
model(C=100.0)
Valid:         AUC: 0.9174 | AUPR: 0.6495 | Acc: 95.21%
best model(C=1.0)
Valid:         AUC: 0.9178 | AUPR: 0.6507 | Acc: 95.24%
Internal test: AUC: 0.9238 | AUPR: 0.6876 | Acc: 95.30%
External test: AUC: 0.9165 | AUPR: 0.6324 | Acc: 94.86%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8680 | AUPR: 0.5649 | Acc: 89.11%
model(criterion=log_loss)
Valid:         AUC: 0.8669 | AUPR: 0.5711 | Acc: 89.48%
model(criterion=entropy)
Valid:         AUC: 0.8669 | AUPR: 0.5711 | Acc: 89.48%
best model(criterion=gini)
Valid:         AUC: 0.8680 | AUPR: 0.5649 | Acc: 89.11%
Internal test: AUC: 0.8844 | AUPR: 0.6139 | Acc: 90.58%
External test: AUC: 0.8612 | AUPR: 0.5757 | Acc: 90.14%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9133 | AUPR: 0.6571 | Acc: 88.90%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9124 | AUPR: 0.6552 | Acc: 89.04%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9129 | AUPR: 0.6559 | Acc: 89.14%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9126 | AUPR: 0.6590 | Acc: 89.31%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9122 | AUPR: 0.6585 | Acc: 89.19%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9130 | AUPR: 0.6586 | Acc: 89.30%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9126 | AUPR: 0.6590 | Acc: 89.31%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9122 | AUPR: 0.6585 | Acc: 89.19%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9130 | AUPR: 0.6586 | Acc: 89.30%
best model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9133 | AUPR: 0.6571 | Acc: 88

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9183 | AUPR: 0.6554 | Acc: 95.20%
model(criterion=squared_error)
Valid:         AUC: 0.9175 | AUPR: 0.6543 | Acc: 95.21%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9183 | AUPR: 0.6554 | Acc: 95.20%
Internal test: AUC: 0.9261 | AUPR: 0.6981 | Acc: 95.43%
External test: AUC: 0.9230 | AUPR: 0.6667 | Acc: 95.25%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9140 | AUPR: 0.6341 | Acc: 95.12%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9154 | AUPR: 0.6355 | Acc: 95.12%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9161 | AUPR: 0.6354 | Acc: 95.10%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9161 | AUPR: 0.6354 | Acc: 95.10%
Internal test: AUC: 0.9181 | AUPR: 0.6725 | Acc: 95.01%
External test: AUC: 0.9154 | AUPR: 0.6189 | Acc: 94.70%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)